# 模型融合预测与交易决策引擎

---

### **目标**
本 Notebook 是整个项目的**应用终端**。它的核心职责是模拟实盘环境，对指定的股票执行完整的“数据-预测-决策”流程。

### **工作流程**
1.  **环境设置与目标选定**: 导入库，加载配置，并指定今天要进行预测的目标股票。
2.  **加载已训练构件**: 
    - 自动查找并加载目标股票**最新版本**的 LGBM 和 LSTM 模型。
    - 加载对应的 `StandardScaler`。
    - 加载两个模型完整的 IC 历史记录，用于动态权重计算。
3.  **获取最新特征**: 调用数据处理流水线，获取截至**今天**的最新一行特征数据。
4.  **独立模型预测**: 分别使用 LGBM 和 LSTM 模型对最新特征进行预测。
5.  **动态权重融合**: 
    - 基于 IC 历史计算出 LGBM 和 LSTM 当前的动态权重。
    - 对两个模型的预测值（经过 Z-score 标准化）进行加权融合，得到最终的预测信号。
6.  **风险审批与决策输出**: 
    - 将融合后的信号提交给 `RiskManager` 进行最终审批（如重复信号检查）。
    - 根据审批结果，输出明确的交易决策（如：**【批准开仓：买入】** 或 **【信号被拒：重复信号】**）。

## 1. 环境设置与目标选定

In [ ]:
import sys, yaml, json, numpy as np, pandas as pd, joblib, torch
from pathlib import Path
from IPython.display import display

# --- 模块导入 ---
try:
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model.builders.lstm_builder import LSTMModel
    from model.builders.model_fuser import ModelFuser
    from risk_management.risk_manager import RiskManager, OrderStatus
    print("INFO: 项目模型导入成功.")
except ImportError as e:
    print(f"WARNNING: 导入失败: {e}. 正在添加项目根目录...")
    project_root = str(Path().resolve()); sys.path.append(project_root) if project_root not in sys.path else None
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model.builders.lstm_builder import LSTMModel
    from model.builders.model_fuser import ModelFuser
    from risk_management.risk_manager import RiskManager, OrderStatus
    print("INFO: 导入成功.")

# --- 加载配置文件 ---
CONFIG_PATH = 'configs/config.yaml'
try:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f: config = yaml.safe_load(f)
    print(f"SUCCESS: Config loaded from '{CONFIG_PATH}'.")
except FileNotFoundError:
    print(f"ERROR: Config file not found."); config = {}

if config:
    db_path = config.get('global_settings', {}).get('order_history_db', 'order_history.db')
    risk_manager = RiskManager(db_path=db_path)
    print(f"--- INFO: RiskManager 已初始化，使用数据库: {db_path} ---")
else:
    risk_manager = None
    print("WARNNING: Config 为空，RiskManager 未初始化。")

# --- 设定要分析的股票 --- 
TARGET_TICKER = '600519.SH'
stock_info = next((s for s in config.get('stocks_to_process', []) if s['ticker'] == TARGET_TICKER), None)
if stock_info:
    TARGET_KEYWORD = stock_info.get('keyword', TARGET_TICKER)
    print(f"--- 目标股票已设定: {TARGET_KEYWORD} ({TARGET_TICKER}) ---")
else:
    print(f"ERROR: 在配置文件中未找到股票 {TARGET_TICKER} 的信息！")

## 2. 加载已训练构件 (模型, Scaler, IC历史)

In [ ]:
models = {}
scalers = {}
ic_histories = {}
all_components_loaded = False

if stock_info:
    model_dir = Path(config.get('global_settings', {}).get('model_dir', 'models')) / TARGET_TICKER
    models_to_load = config.get('global_settings', {}).get('models_to_train', ['lgbm', 'lstm'])
    
    all_found = True
    for model_type in models_to_load:
        print(f"\n--- 正在加载 {model_type.upper()} 的构件...")
        # 查找最新版本的模型
        model_files = sorted(model_dir.glob(f"{model_type}_model_*.p*t")) # .pkl or .pt
        if not model_files:
            print(f"ERROR: 未找到 {model_type.upper()} 的模型文件。请先运行训练流程。")
            all_found = False; continue
        
        latest_model_file = model_files[-1]
        version_timestamp = latest_model_file.stem.split('_')[-1]
        latest_scaler_file = model_dir / f"{model_type}_scaler_{version_timestamp}.pkl"
        ic_history_file = model_dir / f"{model_type}_ic_history.csv"

        # 加载模型、Scaler 和 IC 历史
        try:
            if model_type == 'lgbm':
                models[model_type] = joblib.load(latest_model_file)
            
            # (修改开始)
            elif model_type == 'lstm':
                # 1. (新增) 查找并加载元数据文件
                latest_meta_file = model_dir / f"{model_type}_meta_{version_timestamp}.json"
                if not latest_meta_file.exists():
                    raise FileNotFoundError(f"未找到 LSTM 的元数据文件: {latest_meta_file}")
                
                with open(latest_meta_file, 'r', encoding='utf-8') as f:
                    lstm_metadata = json.load(f)
                
                input_size = lstm_metadata.get('input_size')
                if not input_size:
                    raise ValueError("元数据文件中缺少 'input_size'。")
                
                # 2. 使用元数据动态实例化模型
                lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
                
                print(f"  - INFO: 从元数据加载 input_size = {input_size}")
                model_instance = LSTMModel(
                    input_size=input_size,  # <-- 使用动态加载的值
                    hidden_size_1=lstm_cfg.get('units_1', 64), 
                    hidden_size_2=lstm_cfg.get('units_2', 32), 
                    dropout=lstm_cfg.get('dropout', 0.2)
                )
                
                # 3. 加载模型权重
                model_instance.load_state_dict(torch.load(latest_model_file))
                model_instance.eval() # 设为评估模式
                models[model_type] = model_instance
            ic_histories[model_type] = pd.read_csv(ic_history_file, index_col=0, parse_dates=True)
            print(f"SUCCESS: 成功加载 {model_type.upper()} 版本 '{version_timestamp}' 的模型、Scaler 和 IC 历史。")
        except FileNotFoundError as e:
            print(f"ERROR: 加载失败，找不到文件: {e}")
            all_found = False
        except Exception as e:
            print(f"ERROR: 加载时发生未知错误: {e}")
            all_found = False

    if all_found and len(models) == len(models_to_load):
        all_components_loaded = True
        print("\n--- 所有必需的模型构件已成功加载！---")
else:
    print("ERROR: 股票信息未定义，无法加载模型。")

## 3. 获取最新特征数据

调用数据流水线，获取截至**今天**的最新特征。我们只关心返回的数据框的**最后一行**。

In [ ]:
# Prophet.ipynb -> "3. 获取最新特征数据"

latest_features = None
historical_sequence_for_lstm = None
full_feature_df_for_risk = None

if all_components_loaded:
    try:
        initialize_apis(config)
        
        # 动态计算需要的历史数据长度
        lstm_params = config.get('default_model_params', {}).get('lstm_params', {})
        seq_len = lstm_params.get('sequence_length', 60)
        
        # 调整数据获取的起始日期，以确保能拿到完整的序列
        # 我们需要一个比 seq_len 更长的历史窗口来确保数据质量和处理 NaN
        required_lookback_days = seq_len + 120 # 例如，多获取约半年的数据作为 buffer

        # 创建一个临时的 config 副本进行修改，避免污染全局 config
        pred_config = config.copy()
        end_date_dt = pd.to_datetime(pred_config['strategy_config']['end_date'])
        required_start_date = (end_date_dt - pd.DateOffset(days=required_lookback_days)).strftime('%Y-%m-%d')
        
        # 确保 earliest_start_date 不会比我们需要的晚
        pred_config['strategy_config']['earliest_start_date'] = min(
            pred_config['strategy_config']['earliest_start_date'],
            required_start_date
        )

        full_feature_df_for_risk = get_full_feature_df(
            TARGET_TICKER, 
            pred_config, # 使用修改后的临时配置
            keyword=TARGET_KEYWORD, 
            prediction_mode=True
        )
        
        # 检查返回的数据是否足够长
        if full_feature_df_for_risk is not None and len(full_feature_df_for_risk) >= seq_len:
            # (修改) 为 LGBM 获取最后一行
            latest_features = full_feature_df_for_risk.iloc[-1:]
            
            # (修改) 为 LSTM 提取真实的最后 N 天数据
            historical_sequence_for_lstm = full_feature_df_for_risk.iloc[-seq_len:]
            
            print(f"SUCCESS: 成功获取 {TARGET_KEYWORD} 的最新特征数据 (日期: {latest_features.index[0].date()})。")
            print(f"  - INFO: 已为 LSTM 准备好长度为 {len(historical_sequence_for_lstm)} 的真实历史序列。")
            display(latest_features)
        else:
            print(f"ERROR: 获取到的数据长度不足 {seq_len}，无法为 LSTM 生成有效的输入序列。")
            
    finally:
        shutdown_apis()
else:
    print("模型构件加载不完整，跳过数据获取。")

## 4. 独立模型预测

In [ ]:
# Prophet.ipynb -> "4. 独立模型预测"

predictions = {}
lgbm_quantile_preds = {}

if latest_features is not None:
    label_col = config.get('global_settings', {}).get('label_column', 'label_return')
    feature_cols = [c for c in latest_features.columns if c != label_col]
    X_latest = latest_features[feature_cols]

    # --- LGBM 预测 (现在预测所有分位数) ---
    if 'lgbm' in models:
        X_scaled_lgbm = scalers['lgbm'].transform(X_latest)
        
        # 循环遍历所有已训练的分位数模型
        for name, model in models['lgbm'].items():
            pred = model.predict(X_scaled_lgbm)[0]
            lgbm_quantile_preds[name] = pred
            # 仍然将中位数作为 lgbm 的主要“点预测”
            if name == 'q_0.5':
                predictions['lgbm'] = pred
        
        print("--- LGBM 分位数预测结果 ---")
        for name, pred in lgbm_quantile_preds.items():
            print(f"  - {name} (预期收益率): {pred:.6f}")

    # --- LSTM 预测 ---
    if 'lstm' in models and historical_sequence_for_lstm is not None:
        # 确保使用正确的特征列
        X_sequence_lstm = historical_sequence_for_lstm[feature_cols]

        # 使用真实的、连续的历史序列进行标准化
        X_scaled_lstm = scalers['lstm'].transform(X_sequence_lstm)
        
        # 将其转换为 LSTM 需要的 3D Tensor ([batch_size, seq_len, num_features])
        # unsqueeze(0) 在最前面增加一个 batch 维度 (batch_size=1)
        X_tensor_lstm = torch.from_numpy(X_scaled_lstm).unsqueeze(0).float() 
        
        with torch.no_grad():
            pred_lstm = models['lstm'](X_tensor_lstm).item()
        predictions['lstm'] = pred_lstm
        print(f"\\nLSTM 原始预测值 (基于真实历史序列): {pred_lstm:.6f}")
else:
    print("最新特征数据不可用，无法进行预测。")

## 5. 动态权重融合

In [ ]:
fused_prediction = None

# --- (新增) 全局 Fuser 实例管理 ---
# 我们在 Notebook 的全局作用域中创建或获取 fuser 实例
# 如果 fuser_instance 还不存在，就创建它
if 'fuser_instance' not in locals():
    print("--- INFO: 首次运行，正在初始化并加载 ModelFuser... ---")
    fuser_instance = ModelFuser(TARGET_TICKER, config)
    fuser_instance.load() # 加载已训练的模型

if 'predictions' in locals() and len(predictions) >= 1: # 即使只有一个模型，也可以继续
    print("\\n--- 开始执行模型融合... ---")
    
    if fuser_instance and fuser_instance.is_trained:
        # 1. 准备输入
        # 我们需要确保 preds_dict 的键与 Fuser 训练时使用的名称完全一致
        required_preds = {f'pred_{m}' for m in fuser_instance.meta_model.feature_names_in_}
        preds_dict = {f'pred_{model_type}': pred for model_type, pred in predictions.items()}
        
        # 2. (核心修复) 正确调用 predict 方法
        # fuser_instance 内部会自己维护平滑所需的历史预测 self.recent_preds
        fused_prediction = fuser_instance.predict(preds_dict)
        
        print(f"\\n融合后的最终预测信号 (已平滑): {fused_prediction:.6f}")
        
        # 3. (可选) 显示 Fuser 内部状态
        if hasattr(fuser_instance.meta_model, 'coef_'):
            coefs = {name: val for name, val in zip(fuser_instance.meta_model.feature_names_in_, fuser_instance.meta_model.coef_)}
            print(f"  - 融合模型权重 -> {coefs}")
        
        print(f"  - INFO: 当前平滑窗口内的历史预测值: {fuser_instance.recent_preds}")
        
    else:
        # 如果 Fuser 不可用，回退到简单平均
        print("WARNNING: ModelFuser 不可用或未训练。回退到对基础模型预测进行简单平均。")
        fused_prediction = np.mean(list(predictions.values()))
        print(f"\\n简单平均后的预测信号: {fused_prediction:.6f}")
else:
    print("模型预测不完整或未执行，无法进行融合。")

## 6. 风险审批与决策输出

In [ ]:
from IPython.display import display, HTML

if 'risk_manager' in locals() and risk_manager is None:
    print("ERROR: RiskManager 未成功初始化，无法进行风险审批。")

elif fused_prediction is not None:
    print("\\n--- 开始执行风险审批与决策生成 ---")
    
    # --- 1. 定义信号与阈值 ---
    signal_threshold = config.get('strategy_config', {}).get('signal_threshold', 0.005)
    direction_str = 'BUY' if fused_prediction > signal_threshold else ('SELL' if fused_prediction < -signal_threshold else 'HOLD')
    trade_price = latest_features['close'].iloc[0] # 假设以最新收盘价作为交易参考价

    # 决策报告的初始状态
    decision_approved = False
    decision_notes = "信号强度未达到开仓阈值。"
    order_id = None

    # --- 2. (核心修复) 提交给 RiskManager 审批 ---
    # 只有在明确有开仓意图时才进行审批
    if direction_str in ['BUY', 'SELL']:
        print(f"INFO: 检测到开仓信号 '{direction_str}'，强度为 {fused_prediction:.4%}, 提交至 RiskManager 审批...")
        
        # 确保 full_feature_df_for_risk 存在
        if 'full_feature_df_for_risk' not in locals() or full_feature_df_for_risk is None:
            decision_notes = "ERROR: 缺少用于风险审批的历史市场数据。"
            print(f"  - {decision_notes}")
        else:
            decision_approved, order_id = risk_manager.approve_trade(
                ticker=TARGET_TICKER,
                direction=direction_str,
                price=trade_price,
                latest_market_data=full_feature_df_for_risk,
                config=config 
            )

            if decision_approved:
                decision_notes = f"信号通过所有风险检查。已创建待处理订单。"
                print(f"  - SUCCESS: {decision_notes} (Order ID: {order_id})")
            else:
                # approve_trade 内部会打印具体拒绝原因 (如重复信号)
                decision_notes = "信号被 RiskManager 拒绝（详情请见上方日志）。"
                print(f"  - FAILED: {decision_notes}")
    else:
        print("INFO: 当前信号为 'HOLD'，无需进行开仓审批。")

    # --- 3. 构建并展示最终的决策报告 ---
    final_direction = "看涨 (BUY)" if direction_str == 'BUY' else ("看跌 (SELL)" if direction_str == 'SELL' else "中性 (HOLD)")
    
    # 根据审批结果确定最终状态
    trade_action = "【批准开仓】" if decision_approved else ("【信号被拒】" if direction_str in ['BUY', 'SELL'] else "【无需操作】")

    report_data = [
        ('基础信息', '股票名称', f"{TARGET_KEYWORD} ({TARGET_TICKER})"),
        ('基础信息', '决策生成时间', pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')),
        ('核心观点', '信号方向', final_direction),
        ('核心观点', '信号强度', f"{fused_prediction:+.4%}"),
        ('最终决策', '交易动作', f"{trade_action} {final_direction if decision_approved else ''}"),
        ('最终决策', '备注', decision_notes),
        ('最终决策', '关联订单ID', order_id if order_id else 'N/A'),
    ]
    report_df = pd.DataFrame(report_data, columns=['类别', '项目', '内容']).set_index(['类别', '项目'])
    
    # --- 4. 美化与显示 ---
    def format_value(s):
        idx = s.name[1]
        if idx == '信号强度 (预期收益)': return [f'{v:+.2%}' for v in s]
        return s
        
    def color_direction(val):
        if '看涨' in str(val): return 'color: red; font-weight: bold'
        if '看跌' in str(val): return 'color: green; font-weight: bold'
        return ''
        
    styled_report = (report_df.style
        .set_caption("投资建议")
        .apply(format_value, axis=1)
        .applymap_index(lambda v: 'font-weight: bold;', level=0)
        .applymap(color_direction, subset=pd.IndexSlice[('核心观点', '投资方向'), :])
        .set_table_styles([ ... ]) # (保留您之前的样式)
    )
    
    display(styled_report)
   
else:
    print("无有效融合信号，无法生成投资建议。")